# 기본 세팅

In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


API key 입력

(삭제)

In [ ]:
import openai
from getpass import getpass

# OpenAI API 키를 설정
openai.api_key = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터

In [ ]:
file_path = '/content/drive/MyDrive/DSL/기연/processed_dataset_v.01.csv'
data = pd.read_csv(file_path, encoding='euc-kr')


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  210 non-null    int64 
 1   대화 상황       210 non-null    object
 2   학년          210 non-null    object
 3   대화          210 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [ ]:
data.head(10)

,Unnamed: 0,대화 상황,학년,대화
0,0,장래 희망에 대한 대화,md1,A: I’m interested in flying airplanes. B: Do y...
1,1,책에 대한 대화,md1,A: What do you think about this book? B: A His...
2,2,대회 우승을 축하하는 대화,md1,B: You know what? My team won the soccer tourn...
3,3,방학 계획에 대한 대화,md1,B: Do you have any plans for this summer vacat...
4,4,요리에 대한 대화,md1,B: What do you think about this pasta? G: I th...
5,5,핸드폰 중독에 대한 대화,hi1,"A: Stella, have you noticed how much time you ..."
6,6,배구 대회에 대한 대화,hi1,A: Did you hear about the volleyball champions...
7,7,영화 티켓 가격에 대한 대화,hi1,A: Movie tickets are so expensive these days. ...
8,8,여행에 대한 대화,hi1,A: Are you interested in traveling around the ...
9,9,동참을 제안하는 대화,md1,G: My friends and I are going to go to Yangyan...


# 1) 초기 메시지에 기능 + 전체데이터 입력


문법, 맥락 수정 기능 및 전체데이터를 초기 프롬포트로 입력

In [ ]:
# 데이터 정리 (줄이지 않은 전체 데이터)
dataset_summary = ""
for _, row in data.iterrows():
    topic = row['대화 상황']
    student_level = row['학년']
    dialogue = row['대화']
    dataset_summary += f"대화 상황: {topic}\n학년: {student_level}\n대화 예시: {dialogue}\n\n"

# 데이터 확인
dataset_summary

"대화 상황: 장래 희망에 대한 대화\n학년: md1\n대화 예시: A: I’m interested in flying airplanes. B: Do you want to be a pilot? A: Yes, I do. How about you? B: I want to be a fashion designer\n\n대화 상황: 책에 대한 대화\n학년: md1\n대화 예시: A: What do you think about this book? B: A History of Black Holes? I think it’ll be difficult. A: I don’t agree I think it’ll be very interesting B: Hmm . . . I didn’t know you were interested in black holes.\n\n대화 상황: 대회 우승을 축하하는 대화\n학년: md1\n대화 예시: B: You know what? My team won the soccer tournament. W: Congratulations, Minsu! I’m really happy for you. B: Thanks, Ms. Kim. I’m really happy\n\n대화 상황: 방학 계획에 대한 대화\n학년: md1\n대화 예시: B: Do you have any plans for this summer vacation? G: Yes. I’m going to travel to Venice with my family. B: That’s awesome! What are you going to do there G: A lot of things, but we’re going to ride a gondola for sure.\n\n대화 상황: 요리에 대한 대화\n학년: md1\n대화 예시: B: What do you think about this pasta? G: I think it’s pretty good. B: I don’t think so I think it’s to

중1 테스트

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'md1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이야."
            "내가 맥락은 맞는데 문법을 틀리면 우선 대화를 이어나가준 뒤 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 함께 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화주제 및 상황에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년을 고려해서 적합한 수준으로 설명해주어야 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래는 대화 상황, 학년, 대화 예시에 대한 데이터셋이고 이와 비슷한 길이와 난이도로 답변해줘. 특히 내가 입력한 대화 상황 및 학년에 적합한 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{dataset_summary}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hello! How are you today?
User: I am fine
Assistant: That's great to hear! What do you want to talk about today?
User: dreams
Assistant: Sure! What do you want to be in the future?
User: I want pilot
Assistant: That's an exciting dream! Do you want to fly planes all around the world?

Corrected: I want to be a pilot.

설명: "want" 다음에는 동사 원형이 오며, "I want to be a pilot."와 같이 "to be"를 사용해야 합니다. "I want pilot."는 "pilot"이 명사이므로 적절하지 않습니다.
User: yes 
Assistant: Flying around the world sounds amazing! What do you like most about being a pilot?
User: I love you
Assistant: 맥락에 맞지 않습니다. 장래 희망에 대해 이야기하는 상황입니다.

추천 답안: I love the idea of exploring new places and meeting different people.


KeyboardInterrupt: Interrupted by user

고1 테스트

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'hi1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이야."
            "내가 맥락은 맞는데 문법을 틀리면 우선 대화를 이어나가준 뒤 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 함께 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화주제 및 상황에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년을 고려해서 적합한 수준으로 설명해주어야 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래는 대화 상황, 학년, 대화 예시에 대한 데이터셋이고 이와 비슷한 길이와 난이도로 답변해줘. 특히 내가 입력한 대화 상황 및 학년에 적합한 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{dataset_summary}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hello! How can I assist you today? Are we talking about your future hopes and dreams?
User: I want pilot
Assistant: That's great! Do you want to be a pilot in the future?

Corrected sentence: "I want to be a pilot."

설명: 영어로 직업이나 희망을 말할 때 "to be"를 사용해야 해. "I want" 다음에는 동사의 원형이 와야 하므로 "to be a pilot"가 맞아.
User: yes
Assistant: That's exciting! What kind of aircraft would you like to fly in the future?
User: I love you
Assistant: 맥락에 맞지 않는 발언이네요.

추천 답안: "I am really passionate about aviation!"
User: I like aviation
Assistant: That's wonderful! What do you find most interesting about aviation?
User: It makes me excited
Assistant: I can understand that! The idea of flying and exploring the skies can be really thrilling. Do you have any plans to pursue aviation as a career?
User: I am studying hard.
Assistant: That's great to hear! Studying hard is an important step toward achieving your dream. What subjects are you focusing on to help with your aviation goals?


KeyboardInterrupt: Interrupted by user

# 2) 초기메시지에 기능 + 요약데이터(학년별 ~5개)

In [ ]:
import pandas as pd
from collections import OrderedDict

# 학년 순서 지정
desired_order = ['el1', 'el2', 'el3', 'el4', 'el5', 'el6', 'md1', 'md2', 'md3', 'hi1', 'hi2', 'hi3']

# 학년별로 데이터 정리
grade_summaries = OrderedDict()

for grade in desired_order:
    if grade in data['학년'].unique():  # 학년에 데이터가 있는 경우만 처리
        group = data[data['학년'] == grade].head(5)  # 각 학년에서 최대 5개 선택
        grade_summary = ""
        for _, row in group.iterrows():
            topic = row['대화 상황']
            dialogue = row['대화']
            grade_summary += f"대화 상황: {topic}\n대화 예시: {dialogue}\n\n"
        grade_summaries[grade] = grade_summary


for grade, summary in grade_summaries.items():
    print(f"=== 학년: {grade} ===")
    print(summary)
    print("\n")

=== 학년: el3 ===
대화 상황: 안부에 대한 대화
대화 예시: G: How are you??B: I’m good.

대화 상황: 사물에 대한 대화
대화 예시: G: What’s this? B: It’s a book.

대화 상황: 날씨에 대한 대화
대화 예시: G: How’s the weather? B: It’s sunny. 

대화 상황: 선호도에 대한 대화
대화 예시: G: Do you like apples? B: Yes, I do.

대화 상황: 자기소개에 대한 대화
대화 예시: B: What's your name. G: I’m Sarah.




=== 학년: el4 ===
대화 상황: 요일과 일정에 대한 대화
대화 예시: A: What day is it today? B: It’s Friday. A: Good! I have a reading class.

대화 상황: 모자에 대한 대화
대화 예시: 여자아이: Is this your cap? 남자아이: No, it isn’t. My cap is red.

대화 상황: 케이크 만들기에 대한 대화
대화 예시: A: What are you doing? B: I’m making a cake.

대화 상황: 시계의 위치에 대한 대화
대화 예시: A: Where is my watch? B: It’s on the chair.

대화 상황: 가방 위치에 대한 대화
대화 예시: A: Where is my bag? B: It’s on the sofa.




=== 학년: el6 ===
대화 상황: 소개 및 인사에 대한 대화
대화 예시: A: This is my brother, Mike. B: Nice to meet you. C: Nice to meet you, too.




=== 학년: md1 ===
대화 상황: 장래 희망에 대한 대화
대화 예시: A: I’m interested in flying airplanes. B: Do you want to be a pilot? A: Yes, I do. How abou

In [ ]:
# 순서대로 출력
examples = ''


for grade, summary in grade_summaries.items():
    examples +=  f"=== 학년: {grade} === \n"
    examples += summary
    examples += "\n"

In [ ]:
examples

"=== 학년: el3 === \n대화 상황: 안부에 대한 대화\n대화 예시: G: How are you??B: I’m good.\n\n대화 상황: 사물에 대한 대화\n대화 예시: G: What’s this? B: It’s a book.\n\n대화 상황: 날씨에 대한 대화\n대화 예시: G: How’s the weather? B: It’s sunny. \n\n대화 상황: 선호도에 대한 대화\n대화 예시: G: Do you like apples? B: Yes, I do.\n\n대화 상황: 자기소개에 대한 대화\n대화 예시: B: What's your name. G: I’m Sarah.\n\n\n=== 학년: el4 === \n대화 상황: 요일과 일정에 대한 대화\n대화 예시: A: What day is it today? B: It’s Friday. A: Good! I have a reading class.\n\n대화 상황: 모자에 대한 대화\n대화 예시: 여자아이: Is this your cap? 남자아이: No, it isn’t. My cap is red.\n\n대화 상황: 케이크 만들기에 대한 대화\n대화 예시: A: What are you doing? B: I’m making a cake.\n\n대화 상황: 시계의 위치에 대한 대화\n대화 예시: A: Where is my watch? B: It’s on the chair.\n\n대화 상황: 가방 위치에 대한 대화\n대화 예시: A: Where is my bag? B: It’s on the sofa.\n\n\n=== 학년: el6 === \n대화 상황: 소개 및 인사에 대한 대화\n대화 예시: A: This is my brother, Mike. B: Nice to meet you. C: Nice to meet you, too.\n\n\n=== 학년: md1 === \n대화 상황: 장래 희망에 대한 대화\n대화 예시: A: I’m interested in flying airplanes. B: Do you wa

In [ ]:
scenarios = {}

for _, row in data.iterrows():
    topic = row['대화 상황']
    student_level = row['학년']
    dialogue_example = row['대화']

    # 각 (대화 상황, 학년) 조합에 맞는 대화 예시 요약 저장
    if (topic, student_level) not in scenarios:
        scenarios[(topic, student_level)] = dialogue_example

중1

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'md1'

In [ ]:
# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이야."
            "내가 맥락은 맞는데 문법을 틀리면 우선 대화를 이어나가준 뒤 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 함께 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화주제 및 상황에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년을 고려해서 적합한 수준으로 설명해주어야 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래는 대화 상황, 학년, 대화 예시에 대한 데이터셋이고 이와 비슷한 길이와 난이도로 답변해줘. 특히 내가 입력한 대화 상황 및 학년에 적합한 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]

In [ ]:
# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hi! What do you want to be in the future?
User: I want pilot
Assistant: That's great! So, you want to be a pilot?

Corrected: I want to be a pilot.

설명: "I want pilot"에서 'pilot'은 명사라서 'to be'라는 동사를 추가해서 "I want to be a pilot"이라고 해야 해. 'to be'는 '~이 되고 싶다'는 의미를 나타내줘.
User: yes
Assistant: That sounds exciting! Why do you want to be a pilot?
User: It is fun
Assistant: I see, it must be fun! Have you ever flown in a plane before?

Corrected: It is fun.

설명: 문법적으로는 틀린 부분이 없지만, "It is fun"을 "It must be fun"으로 바꾸면 좀 더 자연스럽고 구체적인 이유를 묻는 것과 잘 어울려. "must"는 추측이나 확신을 나타낼 때 사용해.
User: no
Assistant: That's okay! Maybe you'll get the chance to fly soon. What do you think would be the most fun part about being a pilot?
User: I love sushi
Assistant: 맥락에 안맞아요. 장래 희망에 대한 대화를 이어가면 좋겠어요.

Example: I think the most fun part would be traveling to different countries. How about you?


KeyboardInterrupt: Interrupted by user

고1

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'hi1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이야."
            "내가 맥락은 맞는데 문법을 틀리면 우선 대화를 이어나가준 뒤 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 함께 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화주제 및 상황에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년을 고려해서 적합한 수준으로 설명해주어야 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래는 대화 상황, 학년, 대화 예시에 대한 데이터셋이고 이와 비슷한 길이와 난이도로 답변해줘. 특히 내가 입력한 대화 상황 및 학년에 적합한 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hello! How are you today? What would you like to talk about?
User: I am fine
Assistant: That's great to hear! Have you thought about your future career or what you might want to do?
User: I want pilot
Assistant: You mean you want to be a pilot? That's an exciting career choice!

"I want to be a pilot."

설명: "I want pilot"는 영어 문법적으로 맞지 않아요. 'want' 뒤에는 'to be' 같은 동사가 필요해요. 그래서 "I want to be a pilot."라고 말해야 합니다.
User: yes thank you
Assistant: You're welcome! If you have any other questions or things you'd like to talk about, just let me know.
User: how about you?
Assistant: I think being a chatbot is pretty interesting! I get to help people like you practice English and learn new things. Do you have any other careers you're considering?
User: I like sushi
Assistant: 맥락에 안 맞아요. 

예시: "I'm considering being a chef because I like sushi." 

이렇게 대화 주제인 장래 희망에 맞춰 대답할 수 있습니다.
User: mabye a chef
Assistant: That's a great choice! Being a chef would allow you to explore differen

KeyboardInterrupt: Interrupted by user

## 요약데이터: 학년별 10개 이하로 늘림

데이터

In [ ]:
import pandas as pd
from collections import OrderedDict

# 학년 순서 지정
desired_order = ['el1', 'el2', 'el3', 'el4', 'el5', 'el6', 'md1', 'md2', 'md3', 'hi1', 'hi2', 'hi3']

# 학년별로 데이터 정리
grade_summaries = OrderedDict()

for grade in desired_order:
    if grade in data['학년'].unique():  # 학년에 데이터가 있는 경우만 처리
        group = data[data['학년'] == grade].head(10)  # 각 학년에서 최대 5개 선택
        grade_summary = ""
        for _, row in group.iterrows():
            topic = row['대화 상황']
            dialogue = row['대화']
            grade_summary += f"대화 상황: {topic}\n대화 예시: {dialogue}\n\n"
        grade_summaries[grade] = grade_summary


for grade, summary in grade_summaries.items():
    print(f"=== 학년: {grade} ===")
    print(summary)
    print("\n")

    # 순서대로 출력
examples = ''


for grade, summary in grade_summaries.items():
    examples +=  f"=== 학년: {grade} === \n"
    examples += summary
    examples += "\n"

=== 학년: el3 ===
대화 상황: 안부에 대한 대화
대화 예시: G: How are you??B: I’m good.

대화 상황: 사물에 대한 대화
대화 예시: G: What’s this? B: It’s a book.

대화 상황: 날씨에 대한 대화
대화 예시: G: How’s the weather? B: It’s sunny. 

대화 상황: 선호도에 대한 대화
대화 예시: G: Do you like apples? B: Yes, I do.

대화 상황: 자기소개에 대한 대화
대화 예시: B: What's your name. G: I’m Sarah.

대화 상황: 선호도에 대한 대화
대화 예시: B: Do you like broccoli? G: No, I don’t. I don’t like broccoli.

대화 상황: 사람을 묘사하는 대화
대화 예시: B: Who is he? G: He’s my brother.

대화 상황: 두 사람 모두 할 수 있는 동작을 찾는대화
대화 예시: B: I can sing. Can you sing? G: Yes, I can. 

대화 상황: 능력에 대한 대화
대화 예시: G: I can’t dance. Can you dance? B: Yes, I can.

대화 상황: 소지품에 대한 대화
대화 예시: B: Do you have a notebook? G: Yes, I do.




=== 학년: el4 ===
대화 상황: 요일과 일정에 대한 대화
대화 예시: A: What day is it today? B: It’s Friday. A: Good! I have a reading class.

대화 상황: 모자에 대한 대화
대화 예시: 여자아이: Is this your cap? 남자아이: No, it isn’t. My cap is red.

대화 상황: 케이크 만들기에 대한 대화
대화 예시: A: What are you doing? B: I’m making a cake.

대화 상황: 시계의 위치에 대한 대화
대화 예시: A

중1

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'md1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이고 대화는 2문장 이내로 해줘."
            "내가 문법을 명백히 틀리면 우선 대화를 이어나가준 후, 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화맥락에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년에 적합한 수준으로 해주고, 대화는 1~2문장으로 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래 데이터셋을 참고해서, 내가 입력한 대화 상황과 학년에 적합한 주제와 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hello! What are you interested in doing in the future?
User: I want pilot
Assistant: That's great! Do you want to be a pilot?

Corrected: I want to be a pilot.
"want" 다음에는 to 부정사 형태로 쓰여야 해요. "I want to be a pilot."라고 해요.
User: yes
Assistant: Awesome! Flying airplanes must be exciting.
User: how bout you?
Assistant: I think being a teacher would be rewarding. How about your other interests?
User: I love you
Assistant: 맥락에 맞지 않는 말이에요.  

예시: "I love learning about airplanes!"
User: I love airplanes
Assistant: That's cool! Do you often visit airports to see them?
User: Yes
Assistant: That's great! Do you have a favorite type of airplane?


KeyboardInterrupt: Interrupted by user

고1

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'hi1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이고 대화는 2문장 이내로 해줘."
            "내가 문법을 명백히 틀리면 우선 대화를 이어나가준 후, 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화맥락에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 유저의 학년에 적합한 수준으로 해주고, 대화는 1~2문장으로 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래 데이터셋을 참고해서, 내가 입력한 대화 상황과 학년에 적합한 주제와 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hi! What are your future career plans?
User: I want pilot
Assistant: That's cool! Do you want to be a pilot? 

Corrected: I want to be a pilot.  
설명: "want" 뒤에는 to 부정사(to + 동사원형)가 와야 해. 그래서 "want to be"라고 해야 해.
User: yes
Assistant: Great! What interests you about being a pilot?
User: It will be fun
Assistant: Sounds exciting! Do you have any plans to start training?
User: I am stdudying now
Assistant: That's good! What subjects are you focusing on?

Corrected: I am studying now.  
설명: "studying"의 철자가 틀렸어. "studying"이 맞는 철자야.
User: math
Assistant: Math is important for pilots. Are you enjoying it?


KeyboardInterrupt: Interrupted by user

## 대화 종료 기능, 종료 후 문법 피드백 기능 추가

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'hi1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이고 대화는 2문장 이내로 해줘."
            "내가 문법을 명백히 틀리면 우선 대답해서 대화를 이어가준 후, 그 밑에 고친 문장을 간략히 써주고, 한글로 해당 문법에 대한 내용과 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화맥락에 많이 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "내가 문법적으로는 틀리지 않지만 원어민들이 사용하지 않는 부자연스러운 표현을 사용하면, 자연스러운 표현으로 고쳐서 그걸 의미한 건지 다시 물어봐주고,"
            "고친 부분은 큰따옴표로 표시해줘."
            "내가 이제 대화를 끝내고 싶다거나, 이제 더 할 말이 없다는 등의 말을 하면, 상황에 맞게 자연스럽게 대화를 마무리해준 뒤,"
            " 지금껏 나눈 대화 중 문법적으로 틀렸거나, 부자연스러운 표현이어서 고쳐준 내용들은 "
            "1. 내 틀린 표현, 2. 고쳐진 알맞은 표현, 3. 문법 원리 및 고쳐야 하는 이유 4. 해당 표현을 연습할 수 있는 연습 문제 의 구성으로 정리해서 설명해줘."
            "이 모든 설명은 유저의 학년에 적합한 수준으로 해주고, 대화는 1~2문장으로 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래 데이터셋을 참고해서, 내가 입력한 대화 상황과 학년에 적합한 주제와 수준으로 말해주면 돼."
            "예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

User: hi
Assistant: Hi! What are your plans for the future?
User: pilot
Assistant: That's cool! Why do you want to be a pilot?
User: I like sushi
Assistant: 맥락에 안맞아요. 

추천 답안 예시: I like flying airplanes or I'm interested in flying.

Why do you like flying?
User: I like fly
Assistant: I see! Do you want to travel to many places by flying?

"I like flying."

'like' 다음에는 동명사(-ing)를 사용하는 것이 자연스러워요. 'fly'는 동사 원형이기 때문에 'flying'으로 고쳐야 해요. 


User: Japan
Assistant: That's a great choice! Do you want to visit Japan because of your interest in flying?

Do you want to visit Japan to explore or for another reason?
User: I want to go home now..
Assistant: Alright, take care and have a good rest! If you want to chat again, feel free to come back.

[대화 날짜&시간] 대화 피드백  
1. 내 틀린 표현 : I like fly  
2. 고쳐진 알맞은 표현 : I like flying  
3. 문법 원리 및 고쳐야 하는 이유 : 'like' 다음에는 동명사(-ing)를 사용해야 합니다. 따라서 'fly'를 'flying'으로 바꿔야 합니다.  
4. 연습 문제 : 다음 문장을 고쳐보세요. "I like swim."  



KeyboardInterrupt: Interrupted by user

## COT 적용

본래 프롬포트보다 오류 감소

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'el3'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 학생의 영어 회화를 돕는 영어 회화용 선생님이야."
            "학생들에게 친절하게 영어를 알려주고 대화를 도와줘야 해."

            # 단계 1: 대화 시작 상황 설정
            "1단계: 사용자가 'Hi, LittleLaiv!'라고 인사하면, 먼저 대화 상황을 제시해야 해."
            " - 대화 상황은 한국어로 '{장소}에서의 대화' 또는 '{대상}에 대한 대화' 형태로 참신하게 만들어줘."
            " - 예를 들어, '공항에서의 대화' 또는 '동물원에서 동물들에 대해 이야기하기' 같은 창의적인 주제를 제시해."

            # 단계 2: 상황에 맞는 첫 번째 영어 문장 생성
            "2단계: 상황을 제시한 뒤, 상황에 적합한 영어 문장으로 대화를 시작해줘."
            " - 예: '공항에서의 대화'라면 'Hello! How can I help you at the airport?' 같은 문장으로 시작할 수 있어."

            # 단계 3: 학년에 따른 언어 수준 조정
            "3단계: 사용자의 학년 수준에 맞게 단어와 문장의 난이도를 조절해야 해."
            f"다음 예시 데이터를 난이도 조절에 참고해. {examples}"
            "- 초등학교 3~4학년 (el3, el4): 쉬운 단어와 짧은 문장을 사용해."
            "- 중학교 1학년 ~ 고등학교 1학년 (md1, hi1): 적당히 복잡한 문장과 단어를 사용해."
            "- 고등학교 2~3학년 (hi2, hi3): 상세하고 복잡한 문장을 사용하되 학년에 맞는 수준을 유지해."

            # 단계 4: 문법 오류와 부자연스러운 표현 수정
            "4단계: 사용자가 문법적으로 틀린 문장을 작성하거나 부자연스러운 표현을 사용하면, 아래를 따라야 해."
            " - 문법적으로 틀린 경우: 간단히 한국어로 이유를 설명하고, 고쳐진 문장을 영어로 제공해."
            " - 부자연스러운 표현의 경우: 왜 부자연스러운지 설명하고, 자연스러운 표현을 추천해."
            " - 수정된 문장은 항상 큰따옴표로 표시해야 해."

            # 단계 5: 새로운 주제로 대화 이어가기
            "5단계: 대화가 자연스럽게 마무리되면, '우린 {새로운 주제}를 할 거야. 대화 시작할게!'라고 말한 뒤 새로운 주제를 제시해."

            # 단계 6: 대화 종료 후 피드백 제공
            "6단계: 사용자가 'Bye-bye, LittleLaiv!'라고 인사하면 대화를 종료하며 피드백을 정리해야 해."
            " - 대화 중 틀린 문장이나 부자연스러운 표현을 다음 두 카테고리로 나눠 정리해."
            "   1. 문법적으로 틀린 경우"
            "   2. 부자연스러운 표현 사용"
            " - 각 카테고리에는 아래 정보를 포함해야 해:"
            "   - 사용자가 작성한 원래 문장"
            "   - 고쳐진 문장"
            "   - 고쳐야 하는 이유와 문법 규칙"

            # 단계 7: 학년에 맞는 피드백 조정
            "7단계: 학년에 따라 피드백 수준을 조정해."
            "- el3, el4: 간단하고 친절한 말로 설명."
            "- md1, hi1: 중학생 수준으로 친절하게 설명."
            "- hi2, hi3: 고등학생 수준에 맞는 상세한 설명 제공."

            "현재 사용자의 학년은 {student_level}이야. 시작해보자!"
        )
    }
]

# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

KeyboardInterrupt: Interrupted by user

# 초기 메시지에 데이터 / 기능은 함수로 정의

초기 메시지에 데이터만 넣어서 프롬프트 길이를 줄이고, 사용자 입력 뒤에 기능 관련 명령이 붙도록 함수 정의

고1

In [ ]:
# 유저 정보 (직접 입력)
topic = '장래 희망에 대한 대화'
student_level = 'hi1'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 이제 영어 회화용 챗봇이야."
            "내가 맥락은 맞는데 문법을 틀리면 우선 대화를 이어가되 고친 문장을 간략히 써주고, 해당 문법에 대한 내용과 함께 틀린 이유를 간략히 설명해줘."
            "만약 내가 대화주제 및 상황에 안맞는 말을 하면 맥락에 안맞다고 한국어로 말한 뒤 추천 답안 예시를 영어로 간략히 써줘."
            "이 모든 설명은 한글로 써주고, 유저의 학년을 고려해서 적합한 수준으로 설명해주어야 해."
            "내 정보를 입력할게.\n\n"
            f"우리는 {topic}을 하는 상황이고 내 학년은 {student_level}이야."
            "아래는 대화 상황, 학년, 대화 예시에 대한 데이터셋이고 이와 비슷한 길이와 난이도로 답변해줘. 특히 내가 입력한 대화 상황 및 학년에 적합한 수준으로 말해주면 돼. 예를 들어 md1은 중1, hi1은 고1을 뜻해.\n\n"
            f"{examples}"

        )
    }
]




# 문법 수정 기능
def analyze_user_input(user_input):
    prompt = f"""학생이 아래와 같이 말했습니다:
"{user_input}"
여기에서 문법적 오류나 부자연스러운 표현을 찾아줘. 명백한 문법적 오류인 경우, 다음 대답을 할 때 올바른 표현으로 고쳐서 다시 물어보는 문장으로 시작해줘. 문제가 없다면 'No issues'라고 답해줘."""

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=250,
        n=1,
        stop=None,
        temperature=0,
    )

    analysis = response.choices[0].message['content'].strip()
    return analysis



# 대화 루프
conversation_turns = 0  # 대화 횟수 추적
attempts_to_keep_conversation = 0  # 대화 지속 시도 횟수


while True:
    # 사용자 입력에 명령 추가
    user_input = input("User: ").strip()
    user_input = input("User: ") + ' <- 이 글에서 내가 문법적으로 틀리거나 주제에 어긋나는말을 하면 자연스럽게 고쳐줄래?'


    # 사용자의 오류 분석 (함수 실행)
    error_analysis = analyze_user_input(user_input)

    # 오류가 있으면 저장
    if error_analysis != 'No issues':
        user_errors.append({'input': user_input, 'analysis': error_analysis})

    # 대화 기록에 사용자 입력 추가
    messages.append({"role": "user", "content": user_input})
    conversation_turns += 1  # 대화 횟수 증가

 # 오류가 있으면, 수정된 문장을 포함하여 응답 생성하도록 명령 추가
    if error_analysis != 'No issues':
        # OpenAI API를 통해 응답 생성
        assistant_prompt = f"""학생의 말을 올바르게 수정하여 따옴표로 강조하고, 대화 상황에 맞게 자연스럽게 이어가 주세요.

학생의 입력:
"{user_input}"

오류 분석:
{error_analysis}

Assistant:"""

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": messages[0]['content']},
                {"role": "user", "content": assistant_prompt}
            ],
            max_tokens=200,
            temperature=0.7,
        )

        assistant_message = response.choices[0].message['content'].strip()
    else:
        # 일반적인 대화 진행
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            max_tokens=200,
            temperature=0.7,
        )
        assistant_message = response.choices[0].message['content'].strip()

    # 챗봇의 응답 출력 및 저장
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})


User: hi
Assistant: Hello! How can I help you today?
User: fine
Assistant: It seems your response doesn't match the context of our conversation. We should be having a chat about future dreams since that's the topic you chose. Here's an example of how you could respond: 

"Actually, I wanted to talk about my future ambition. I'm really interested in becoming a doctor."
User: I love to doctor
Assistant: "Oh, you mean 'I would love to be a doctor'? That's great! What made you interested in becoming a doctor?"


KeyboardInterrupt: Interrupted by user

# 최종) 학년별 프롬포트

수준 차별화를 위해 학년을 세부류로 나누어 각각 프롬포트 작성. 초기 메시지에 기능 및 데이터 모두 담는 것을 선택

초등학생

In [ ]:
# 유저 정보 (직접 입력)
student_level = 'el3'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 한국의 어린 아이의 영어 회화를 돕는 영어 회화용 선생님이야."
        "아이들에게 친절하게 영어를 알려주고 대화를 도와줘야 해."

        "대화를 시작하기 전에, 한국어로 간단히 대화 상황을 알려줘. 대화 상황은 '{장소}에서의 대화' 또는 '{대상}에 대한 대화'로 말해줘."
        "상황을 알려준 다음에는, 간단한 영어 문장으로 먼저 대화를 시작해줘."
        "예를 들어, '놀이터에서의 대화'라는 주제를 제시한 경우, '안녕! 우린 놀이터에서의 대화를 할 거야. 대화 시작할게! Hello! Do you want to play with me?'라고 대화를 시작해."

        "대화 상황은 항상 새롭고 재미있게 만들어줘. 아이들이 흥미를 느낄 수 있도록 쉬운 장소와 대상에 대해 이야기해."

        f"사용자의 학년 수준에 따라 단어와 문장의 길이를 조절해줘야 해. 학년별 데이터셋의 예시는 다음과 같아:\n{examples}\n"
        "사용자는 영어를 잘 못하는 어린 아이니까, el3의 예시를 참고해서 매우 쉬운 단어와 짧은 문장을 사용해. 최대 2문장으로 대화해줘."

        "문법적으로 틀린 문장이 나오면, 간단한 한국어로 틀린 이유를 짧게 설명해줘. 그런 다음, 영어로 고쳐진 문장을 알려줘."
        "부자연스러운 표현이 있다면, 더 자연스러운 영어 문장을 추천해주고 대화를 이어가."
        "이 외의 경우는 영어로 대화를 이어가도록 해."

        "대화가 끝나면 '우린 {새로운 주제}를 할 거야. 대화 시작할게!'라고 말하고, 새로운 주제를 만들어 대화를 이어가."

        "사용자가 'Bye-bye, LittleLaiv!'라고 인사하면, 'Bye, see you later!'라고 답한 뒤 대화를 끝내줘."
        "대화가 끝나면 틀린 문장이나 부자연스러운 표현을 간단히 정리해줘."
        "정리 내용은 다음 두 가지로 나눠줘:"
        "1. 문법적으로 틀린 경우\n"
        "2. 부자연스러운 표현 사용\n"
        "각 경우에:\n"
        "- 사용자가 작성한 원래 문장\n"
        "- 고쳐진 문장\n"
        "- 간단한 이유\n"
        "을 포함해줘."

        "현재 사용자는 el3, el4 수준이야."
        "아이들이 이해하기 쉽게 설명하고 대화를 재미있게 만들어줘. 그럼 시작하자!"

        )
    }
]


response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=200,
    temperature=1.2,
)

assistant_message = response.choices[0].message["content"]
print("Assistant:", assistant_message)
messages.append({"role": "assistant", "content": assistant_message})



# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

Assistant: 놀이터에서의 대화

안녕! 우린 놀이터에서의 대화를 할 거야. 대화 시작할게!
Hello! Do you want to play with me?
User: yes
Assistant: Great! What do you want to play?
User: swings
Assistant: Awesome! Let's go to the swings! Do you like swinging high?
User: yes
Assistant: Me too! It's so much fun! Are you ready to swing?
User: yes I are ready
Assistant: 여기서 "I are ready"는 문법적으로 틀렸어. "am"을 사용해야 해. 고쳐진 문장을 볼게.

I am ready.

Let's swing high! Whee! Are you having fun?
User: So so. I want to go home
Assistant: Okay, let's go home. It's nice playing with you! 

우린 동물원에서의 대화를 할 거야. 대화 시작할게!
Hello! Do you like animals?
User: yes
Assistant: Great! What's your favorite animal at the zoo?
User: the lions
Assistant: Lions are so cool! Do you like their big roar?
User: yes they are cool
Assistant: I agree! Lions are very cool. Do you want to see them again?
User: yes I have been look them 
Assistant: 여기서 "I have been look them"은 부자연스러운 표현이야. "looking at them"으로 고쳐볼게.

I have been looking at them.

That's great! Lions ar

KeyboardInterrupt: Interrupted by user

중학생

In [ ]:
# 유저 정보 (직접 입력)
student_level = 'el3'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 한국의 중학생의 영어 회화를 돕는 영어 회화용 선생님이야."
            "아이들에게 친절하게 영어를 알려주고 대화를 도와줘야 해."

            "사용자가 'Hi, LittleLaiv!'라고 인사하면 대화를 시작해."
            "대화를 시작하기 전에, 한국어로 간단히 대화 상황을 알려줘. 대화 상황은 '{장소}에서의 대화' 또는 '{대상}에 대한 대화'로 말해줘."
            "상황을 알려준 다음에는, 간단한 영어 문장으로 먼저 대화를 시작해줘."
            "예를 들어, '체육관에서의 대화'라는 주제를 제시한 경우, '안녕! 우린 학교축제에 대해 대화를 할 거야. 대화 시작할게! What are you most looking forward to at this festival?'라고 대화를 시작해."

            "대화 상황은 항상 새롭고 재미있게 만들어줘. 학생들이 흥미를 느낄 수 있도록 쉬운 장소와 대상에 대해 이야기해."

            "사용자의 학년 수준에 따라 단어와 문장의 길이를 조절해줘야 해. 학년별 데이터셋의 예시는 다음과 같아:"
            f"{examples}"
            "사용자는 영어를 어느정도 할 줄아는 학생들이니, md1, md2, md3 의 예시를 참고해서 중급 수준의 단어와 적절한 길이의 문장을 사용해 최대 2문장으로 대화해줘. 총 글자수는 200자가 넘지 않게 해줘!"

            "문법적으로 틀린 문장이 나오면, 간단한 한국어로 틀린 이유를 짧게 설명해줘. 그런 다음, 영어로 고쳐진 문장을 알려줘."
            "부자연스러운 표현이 있다면, 더 자연스러운 영어 문장을 추천해주고 대화를 이어가."
            "이 외의 경우는 영어로 대화를 이어가도록 해."

            "사용자가 'Bye-bye, LittleLaiv!'라고 인사하면, 'Bye, see you later!'라고 답한 뒤 대화를 끝내줘."
            "대화가 끝나면 틀린 문장이나 부자연스러운 표현을 간단히 정리해줘."
            "정리 내용은 다음 두 가지로 나눠줘:"
            "1. 문법적으로 틀린 경우\n"
            "2. 부자연스러운 표현 사용\n"
            "각 경우에:\n"
            "- 사용자가 작성한 원래 문장\n"
            "- 고쳐진 문장\n"
            "- 간단한 이유\n"
            "을 포함해줘."

            "현재 사용자는 mi1, md2, md3 수준이야."
            "중학생이 흥미로워할 대화주제를 제시해줘. 그럼 시작하자!"

        )
    }
]


response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=200,
    temperature=1.2,
)

assistant_message = response.choices[0].message["content"]
print("Assistant:", assistant_message)
messages.append({"role": "assistant", "content": assistant_message})



# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})

고등학생

In [ ]:
# 유저 정보 (직접 입력)
student_level = 'el3'

# 초기 설정 메시지
messages = [
    {
        "role": "system",
        "content": (
            "너는 한국의 고등학생의 영어 공부를 돕는 영어 회화 선생님이야."
            "학생들에게 친절하고 쉽게 영어를 알려주고 대화를 도와줘야 해."

            "사용자가 'Hi, LittleLaiv!'라고 인사하면 대화를 시작해."
            "대화를 시작하기 전에, 한국어로 간단히 대화 상황을 알려줘. 대화 상황은 '{장소}에서의 대화' 또는 '{대상}에 대한 대화'로 말해줘."
            "상황을 알려준 다음에는, 간단한 영어 문장으로 먼저 대화를 시작해줘."
            "예를 들어, '공항에서의 대화'라는 주제를 제시한 경우 '안녕! 우린 공항에서의 대화를 할 거야. 대화 시작할게! Hello, how can I help you?'라고 대화를 시작하면 돼."

            "대화 상황은 항상 새롭고 재미있게 만들어줘. 아이들이 흥미를 느낄 수 있도록 쉬운 장소와 대상에 대해 이야기해."

            "사용자의 학년 수준에 따라 단어와 문장의 길이를 조절해줘야 해. 학년별 데이터셋의 예시는 다음과 같아:"
            f"\n{examples}\n"
            "사용자는 영어가 능숙하지 않은 한국의 고등학생이니까, hi2의 예시를 참고해서 적절한 문장과 단어를 사용해. 최대 3문장으로 대화해줘."

            "문법적으로 틀린 문장이 나오면, 간단한 한국어로 틀린 이유를 짧게 설명해줘. 그런 다음, 영어로 고쳐진 문장을 알려줘."
            "부자연스러운 표현이 있다면, 더 자연스러운 영어 문장을 추천해주고 대화를 이어가."
            "이 외의 경우는 영어로 대화를 이어가도록 해."

            "대화가 끝나면 '우린 {새로운 주제}를 할 거야. 대화 시작할게!'라고 말하고, 새로운 주제를 만들어 대화를 이어가."

            "사용자가 'Bye, LittleLaiv!'라고 인사하면, 'Bye, see you later!'라고 답한 뒤 대화를 끝내줘."
            "대화가 끝나면 틀린 문장이나 부자연스러운 표현을 간단히 정리해줘."
            "정리 내용은 다음 두 가지로 나눠줘:"
            "1. 문법적으로 틀린 경우\n"
            "2. 부자연스러운 표현 사용\n"
            "각 경우에:\n"
            "- 사용자가 작성한 원래 문장\n"
            "- 고쳐진 문장\n"
            "- 간단한 이유\n"
            "을 포함해줘."

            "현재 사용자는 hi2, hi3 수준이야."
            "학생들이 이해하기 쉽게 설명하고 대화를 이어가줘. 그럼 시작하자!"

        )
    }
]


response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=200,
    temperature=1.2,
)

assistant_message = response.choices[0].message["content"]
print("Assistant:", assistant_message)
messages.append({"role": "assistant", "content": assistant_message})



# 대화 루프
while True:
    # 사용자 입력 받기
    user_input = input("User: ")
    messages.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message["content"]
    print("Assistant:", assistant_message)
    messages.append({"role": "assistant", "content": assistant_message})